In [44]:
import pandas as pd 
import json
import re
import psycopg2
import nltk
import nan 
from nltk.corpus import stopwords
from configparser import ConfigParser
from sklearn.feature_extraction.text import TfidfVectorizer

stopwords = set(stopwords.words('english'))

In [60]:
#This function facilitates the connection to the database
def connect():
    try:
        
        parser = ConfigParser()
        parser.read('database.ini')

        #params = config()
        host = parser.get('postgresql','DB_HOST')
        port = parser.get('postgresql','DB_PORT')
        database =parser.get('postgresql','DB_NAME')
        user = parser.get('postgresql','DB_USER')
        password = parser.get('postgresql','DB_PW')

        #params = config()
        #print(host, port, password, user, database)
        conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)

        curr = conn.cursor()
        
#         query = """SELECT table_name from information_schema.tables WHERE table_schema = 'public'"""
#         curr.execute(query)
        
#         rows = curr.fetchall()

#         for row in rows:
#             print(row[0])

        return curr, conn
    except( Exception, psycopg2.DatabaseError) as err:
        print(err)
    


In [58]:
#A function that gets the user table and returns it as a dataframe
def get_users(curr, conn):
    query = "SELECT id FROM users"
    table = pd.read_sql_query(query, conn)
    table = pd.DataFrame(table)
    return table
    print(table)
    #table
    
cur, con = connect()
getUsers(cur, con)
#con.close()
#cur.close()

                                      id                    username
0   c8a93f30-fe1b-4cea-a7e2-49c37e908133                       Louis
1   0593c284-4bcf-4589-aa5a-19d34f665044                       Thato
2   1817e726-76f8-4909-a175-5e623459df52                    Mpendulo
3   4757b4f9-a567-49aa-bfb8-ddcee00218df                        Tana
4   219addc5-102c-405e-9859-5c4e6baf7588                       Vince
5   421b8bf1-d3cf-4285-9132-ed38f63efbde                    Tanatswa
6   a8cc8896-d967-4158-8e7e-ce8623b4ad0c                        Test
7   9ed990de-d9be-4f71-a511-57d26500bf8c                        Jean
8   55c9b00f-6833-4f1f-8141-c0ba887e4d66                         Get
9   37641731-3ccf-4cca-8334-76fbc9ac9411                         Set
10  d5df0e03-c7c1-47d1-bda7-ac698c95aea6  dtanatswa.social@gmail.com
11  a26dd07e-3a9c-498e-90f1-8658eb0d5c2a  tanatswa.dendere@gmail.com
12  ee3f5393-0d27-40da-b0e3-03d573e5e9b0              tana@gmail.com


In [ ]:
def remove_columns_from_products(table):
    #table.drop(['image', 'link', 'price', 'availability', 'retailer', 'details'], axis=1, inplace=True)
    table.dropna(inplace=True)

In [61]:
def get_wishlist(cur, con):
    query = "SELECT * FROM wishilist_cpu"
    cpus = pd.read_sql_query(query, con)
    cpus = pd.DataFrame(cpus)
    
    query = "SELECT * FROM wishilist_gpu"
    gpus = pd.read_sql_query(query, con)
    
    table = cpus.append(gpus)
    #any preprocessing needing to be done
    #table = remove_columns_from_products(table)
    return table 

cur, con = connect()
get_wishlist(cur, con)

could not translate host name "siliconscraper-1.csvt0txubrxm.us-east-2.rds.amazonaws.com" to address: Unknown host



TypeError: cannot unpack non-iterable NoneType object

In [62]:
def get_all_products(cur, con):
    query = "SELECT id, brand, type, description FROM cpus"
    cpus = pd.read_sql_query(query, con)
    cpus = pd.DataFrame(cpus)
    
    query = "SELECT id, brand, type, description FROM gpu"
    gpus = pd.read_sql_query(query, con)
    
    table = cpus.append(gpus)
    
    # any preprocessing needed to be done
    table.dropna(inplace=True)
    
    return table

SyntaxError: unexpected EOF while parsing (<ipython-input-62-fefe4e811369>, line 1)

In [ ]:
#remove all products that are already in a wishlist 
#all_products = all_products.loc[all_products[all_products['id'] != wishlist_products['id']]]


In [54]:
#This function sorts the cpu products by renaming and merging columns for consistency
def filter_cpu(data):
    data_des = data[['type', 'retailer', 'brand','# of CPU Cores', '# of Threads', 'Base Clock', 'Max Boost Clock',
                                        'Default TDP / TDP', 'Processor Base Frequency','Max Turbo Frequency', 'TDP']]
    
    df_change = data_des.loc[data_des['Base Clock'].isnull()]
    data_des.drop(df_change.index, inplace=True)
    df_change.drop(['Base Clock', 'Default TDP / TDP', 'Max Boost Clock'], axis=1, inplace=True)
    df_change.rename({'Processor Base Frequency': 'Base Clock', 'Max Turbo Frequency': 'Max Boost Clock'}, axis=1, inplace=True)
    
    data_des.drop(['Processor Base Frequency', 'Max Turbo Frequency', 'TDP'], axis=1, inplace=True)
    data_des.rename({'Default TDP / TDP': 'TDP'}, axis=1, inplace=True)
    data_des = pd.merge(data_des, df_change, how='outer')
    return data_des


In [ ]:
cur, con = connect()

wishlist_products = get_wishlist(cur, con)
wishlist_products.drop_duplicates(subset='product_id', keep=False, inplace=True)

all_products = get_all_products(cur, con)

In [ ]:
modify = re.compile(r"\([^(),{}]*\)")
def clean_data(text):
    if text == '' or not list(text):
        return math.nan
    final = [(x +"-" + y).lower() for x,y in zip(list(text), list(text.values()))]
    return "  ".join(final)

all_products.dropna(inplace=True)
all_products['descriptions_cl'] = all_products['descriptions'].apply(clean_data)
all_products.drop(['description'], axis=1, inplace=True)
all_products.dropna(inplace=True)

In [ ]:
#vectorize products and calculate similarity 
all_products.set_index('id', inplace=True)
all_products['all_features'] = all_products['brand'] +" "+ all_products['retailer'] +" "+ all_products['description_cl']
vectorizor = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0)
tfidf_matrix = vectorizor.fit_transform(all_products['all_features'])
similarity = linear_kernel(tfidf_matrix, tfidf_matrix)
similarity

In [ ]:
rec = pd.Series(all_products.index)

#function to carry through the recommending 
def recommend(idx, cosine = similarity):
    
    #find the index of the product in the index series
    item = rec[rec == idx].index[0]
    
    #get array of similarities and sort them in descending order
    similarities = list(enumerate(cosine[item]))
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    
    #get the top 5 products, excluding the product itself and products in the users watchlist 
    similarities = similarities[1:6]
    
    #get index of top 5 
    index = [i[0] for i in similarities]
    
    #get the products using the the retrieved index
    return all_products.iloc[index]

In [ ]:
#get recommendations for all products in wishlist 
recs = []
wishlist = list(wishlist_description['id'])
for i, product in enumerate(wishlist):
    response = list(recommend(product).index)
    recs.append(response)
    

result = pd.DataFrame(wishlist_description['id'], columns=['id','recommendations'])
result['recommendations'] = recs
result.set_index('id', inplace=True)

In [ ]:
#calculate loss 

In [ ]:
def main():
    #get users
    cur, con = connect()
    users = get_users(cur, con)
    
    #get users wishlist; filter
    wishlist_products = get_wishlist(cur, con)
    
    #get all products that are not in wishlist; filter
    all_products = get_all_products(cur, con)
    #encode data 
    
    #predict